<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [ ]:
##### Your Code Here #####

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [31]:
##### Your Code Here #####
import tensorflow
import keras

from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()



In [32]:
X_test.shape


(10000, 28, 28)

In [33]:
y_train[:10000].shape

(10000,)

In [34]:

X_train = X_train[:10000]
y_train = y_train[:10000]
X_test = X_test[:2000]
y_test = y_test[:2000]

In [ ]:
from sklearn.datasets import fetch_mldata

X, y = fetch_mldata('mnist_784')

In [ ]:
X

In [ ]:


# normalizing to keep gradients manageable

X = X / 255

# cutting down model size from 70k to 20k

X = X[:20000]
y = y[:20000]

# one hot encoding labels

digits = 10
examples = y.shape[0]

y = y.reshape(1, examples)

Y_new = np.eye(digits)[y.astype('int32')]
y_new = Y_new.T.reshape(digits, examples)

m = 16000
m_test = X.shape[0] - m

X_train, X_test = X[:m].T, X[m:].T
y_train, y_test = y_new[:,:m], y_new[:,m:]

shuffle_index = np.random.permutation(m)
X_train, y_train = X_train[:, shuffle_index], y_train[:, shuffle_index]

In [38]:
# check to make sure images are correct
# looks wrong, but continuing
from matplotlib import pyplot as plt
import matplotlib
i = 12
plt.imshow(X_train.reshape(10000,784), cmap=matplotlib.cm.binary)
plt.axis('off')
plt.show()

In [40]:


def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

def compute_loss(y, y_hat):
    m = y.shape[1]
    L = -(1./m) * (np.sum(np.multiply(np.log(y_hat), y)) + np.sum(np.multiply(np.log(1-y_hat), (1-y))))
    return L

In [42]:
import numpy as np
learning_rate = 1

X = X_train
y = y_train

n_x = X.shape[0]
m = X.shape[1]

W = np.random.randn(n_x, 1) * 0.01
b = np.zeros((1,1))

for i in range(2000):
    Z = np.matmul(W.T, X) + b
    A = sigmoid(Z)
    
    cost = compute_loss(y, A)
    
    dW = (1 / m) * np.matmul(X, (A-y).T)
    db = (1 / m) * np.sum(A - y, axis=1, keepdims=True)
    
    W = W - learning_rate * dW
    b = b - learning_rate * db
    
    if (i % 100 == 0):
        print('epoch', i, 'cost', cost)
        
print('final cost', cost)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 28 is different from 10000)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Z = np.matmul(W.T, X_test) + b
A = sigmoid(Z)

predictions = (A>0.5)[0,:]
labels = (y_test ==1)[0,:]

print(confusion_matrix(predictions, labels))

print(classification_report(predictions, labels))

In [ ]:
X = X_train
Y = y_train

n_x = X.shape[0]
n_h = 64
learning_rate = 1

W1 = np.random.randn(n_h, n_x)
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(1, n_h)
b2 = np.zeros((1, 1))

for i in range(2000):

    Z1 = np.matmul(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.matmul(W2, A1) + b2
    A2 = sigmoid(Z2)

    cost = compute_loss(Y, A2)

    dZ2 = A2-Y
    dW2 = (1./m) * np.matmul(dZ2, A1.T)
    db2 = (1./m) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.matmul(W2.T, dZ2)
    dZ1 = dA1 * sigmoid(Z1) * (1 - sigmoid(Z1))
    dW1 = (1./m) * np.matmul(dZ1, X.T)
    db1 = (1./m) * np.sum(dZ1, axis=1, keepdims=True)

    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1

    if i % 100 == 0:
        print("Epoch", i, "cost: ", cost)

print("Final cost:", cost)



In [ ]:
def compute_multiclass_loss(Y, Y_hat):
    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[1]
    L = -(1/m) * L_sum
    
    return L

n_x = X_train.shape[0]
n_h = 64
learning_rate = 1

W1 = np.random.randn(n_h, n_x)
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(digits, n_h)
b2 = np.zeros((digits, 1))

X = X_train
Y = y_train

for i in range(2000):
    Z1 = np.matmul(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.matmul(W2, A1) + b2
    A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)
    
    cost = compute_multiclass_loss(Y, A2)
    
    dZ2 = A2 - Y
    dW2 = (1./m) * np.matmul(dZ2, A1.T)
    db2 = (1./m) * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.matmul(W2.T, dZ2)
    dZ1 = dA1 * sigmoid(Z1) * (1 - sigmoid(Z1))
    dW1 = (1./m) * np.matmul(dZ1, X.T)
    db1 = (1./m) * np.sum(dZ1, axis=1, keepdims=True)
    
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    
    if (i % 100 == 0):
        print('epoch', i, 'cost:', cost)
        
print('final cost:', cost)

In [297]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
  # create model
        model = Sequential()
        model.add(Dense(38, input_dim=19, activation='relu'))
        model.add(Dense(19, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
  # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

C:\Users\mhdal\Anaconda3\envs\lambda-neural\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.8002271784243894 using {'batch_size': 60, 'epochs': 20}
Means: 0.7961096100029952, Stdev: 0.0013647352184608407 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7905722032736838, Stdev: 0.006227957571291364 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7978134327410095, Stdev: 0.0004931025843609994 with: {'batch_size': 40, 'epochs': 20}
Means: 0.8002271784243894, Stdev: 0.001532294901941202 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7990912961364046, Stdev: 0.0012252929240128077 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7971035080607364, Stdev: 0.0031924955226660494 with: {'batch_size': 100, 'epochs': 20}


In [297]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
  # create model
        model = Sequential()
        model.add(Dense(38, input_dim=19, activation='relu'))
        model.add(Dense(19, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
  # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

C:\Users\mhdal\Anaconda3\envs\lambda-neural\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.8002271784243894 using {'batch_size': 60, 'epochs': 20}
Means: 0.7961096100029952, Stdev: 0.0013647352184608407 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7905722032736838, Stdev: 0.006227957571291364 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7978134327410095, Stdev: 0.0004931025843609994 with: {'batch_size': 40, 'epochs': 20}
Means: 0.8002271784243894, Stdev: 0.001532294901941202 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7990912961364046, Stdev: 0.0012252929240128077 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7971035080607364, Stdev: 0.0031924955226660494 with: {'batch_size': 100, 'epochs': 20}


## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?